In [1]:
import pandas as pd
import os 
import pymongo
import json
from pprint import pprint

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [3]:
db = client.rescue_angels_db

In [4]:
org = db.org_collection

In [5]:
zipcode2_df = pd.read_csv("zip_lat_lon.csv")
zipcode2_df

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Unnamed: 8
0,55795,Willow River,MN,46.317812,-92.84315,-6,1,46.317812,-92.84315
1,45388,Yorkshire,OH,40.328535,-84.47938,-5,1,40.328535,-84.47938
2,39483,Foxworth,MS,31.218509,-89.90761,-6,1,31.218509,-89.90761
3,31503,Waycross,GA,31.205194,-82.37534,-5,1,31.205194,-82.37534
4,45833,Delphos,OH,40.841409,-84.34178,-5,1,40.841409,-84.34178
...,...,...,...,...,...,...,...,...,...
43186,75034,Frisco,TX,33.143792,-96.83938,-6,1,33.143792,-96.83938
43187,41631,Grethel,KY,37.479754,-82.64922,-5,1,37.479754,-82.64922
43188,98362,Port Angeles,WA,48.097144,-123.40628,-8,1,48.097144,-123.40628
43189,83212,Arbon,ID,42.514471,-112.54626,-7,1,42.514471,-112.54626


In [6]:
zipcode3_df = zipcode2_df[["Zip", "Latitude", "Longitude"]]
zipcode3_df

,Zip,Latitude,Longitude
0,55795,46.317812,-92.84315
1,45388,40.328535,-84.47938
2,39483,31.218509,-89.90761
3,31503,31.205194,-82.37534
4,45833,40.841409,-84.34178
...,...,...,...
43186,75034,33.143792,-96.83938
43187,41631,37.479754,-82.64922
43188,98362,48.097144,-123.40628
43189,83212,42.514471,-112.54626


In [7]:
#need to convert datatypes in order to merge dataframes 
convert_zipdict = {'Zip': str}
zipcode3_df = zipcode3_df.astype(convert_zipdict)
print(zipcode3_df.dtypes) 
print(len(zipcode3_df))

Zip           object
Latitude     float64
Longitude    float64
dtype: object
43191


In [8]:
# get mongodb into pandas in order to merge with latlong
#select the collection within the database (i.e. org = db.org_collection as defined above)
#convert entire collection to Pandas dataframe
org_df = pd.DataFrame(list(org.find()))
org_df.head()

,_id,index,Unnamed: 0,id,name,email,phone,url,website,mission_statement,...,hours_saturdsy,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,600c8d8b9129859f3e7d80f4,0,0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,NaN,https://www.petfinder.com/member/us/pa/fairles...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pa912
1,600c8d8b9129859f3e7d80f5,1,1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,NaN,We are a private foster home group. We are NOT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ga762
2,600c8d8b9129859f3e7d80f6,2,2,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oh749
3,600c8d8b9129859f3e7d80f7,3,3,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tx876
4,600c8d8b9129859f3e7d80f8,4,4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mo556


In [9]:
#need to convert datatypes in order to merge dataframes
# convert_petzipdict = {'Zip': str}
# pet_zipcodes_df = pet_zipcodes_df.astype(convert_petzipdict)
# print(pet_zipcodes_df.dtypes) 

convert_orgdict = {'address_postcode': str}
org2_df = org_df.astype(convert_orgdict)
print(org2_df.dtypes) 

_id                       object
index                      int64
Unnamed: 0                 int64
id                        object
name                      object
email                     object
phone                     object
url                       object
website                   object
mission_statement         object
distance                 float64
address_address1          object
address_address2          object
address_city              object
address_state             object
address_postcode          object
address_country           object
hours_monday              object
hours_tuesday             object
hours_wednesday           object
hours_thursday            object
hours_friday              object
hours_saturdsy            object
hours_sunday              object
adoption_policy           object
adoption_url              object
socialmedia_facebook      object
socialmedia_twitter       object
socialmedia_youtube       object
socialmedia_instagram     object
socialmedi

In [10]:
# correct typo in hours_saturdsy
org2_df = org2_df.rename(columns={'hours_saturdsy': 'hours_saturday'})
org2_df

,_id,index,Unnamed: 0,id,name,email,phone,url,website,mission_statement,...,hours_saturday,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,600c8d8b9129859f3e7d80f4,0,0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,NaN,https://www.petfinder.com/member/us/pa/fairles...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pa912
1,600c8d8b9129859f3e7d80f5,1,1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,NaN,We are a private foster home group. We are NOT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ga762
2,600c8d8b9129859f3e7d80f6,2,2,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oh749
3,600c8d8b9129859f3e7d80f7,3,3,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tx876
4,600c8d8b9129859f3e7d80f8,4,4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9639,600c8d8c9129859f3e7da7ff,9995,9995,UT08,Washington City Animal Shelter,abarnson@washingtoncity.org,(435) 673-7194,https://www.petfinder.com/member/us/ut/washing...,NaN,NaN,...,NaN,NaN,We want to get all of our animals at the shelt...,NaN,https://www.facebook.com/washcityanmlshelter/,NaN,NaN,NaN,NaN,ut08
9640,600c8d8c9129859f3e7da800,9996,9996,NJ625,"A Dog's Place Rescue, Inc.",sherman.lisa28@gmail.com,(484) 800-2055,https://www.petfinder.com/member/us/pa/ardmore...,http://adogsplacerescuepa.org,PLEASE NOTE!!! All dogs posted on our site co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nj625
9641,600c8d8c9129859f3e7da801,9997,9997,IN595,Great Babies Rescue Inc.,greatbabiesrescue@yahoo.com,(812) 653-1985,https://www.petfinder.com/member/us/in/paoli/g...,http://greatbabiesrescue.org,"Our mission is simple, to rescue, rehabilitate...",...,NaN,NaN,Great Babies Rescue takes the adoption process...,NaN,NaN,NaN,NaN,NaN,NaN,in595
9642,600c8d8c9129859f3e7da802,9998,9998,NJ679,"Thrifty Kitty, Inc.",thriftykitty@msn.com,(609) 287-9121,https://www.petfinder.com/member/us/nj/oceanvi...,NaN,VOLUNTEER YOUR TIME\r\n\r\n\r\nThrifty Kitty i...,...,NaN,NaN,Thrifity Kitty is no longer adopting cats or k...,NaN,NaN,NaN,NaN,NaN,NaN,nj679


In [11]:
new_zip3_df = zipcode3_df.rename(columns={'Zip': 'address_postcode'})

In [12]:
# change Zip header name to match field in incoming mongodb

new_zip3_df = new_zip3_df.rename(columns={'Zip': 'address_postcode'})
new_zip3_df

,address_postcode,Latitude,Longitude
0,55795,46.317812,-92.84315
1,45388,40.328535,-84.47938
2,39483,31.218509,-89.90761
3,31503,31.205194,-82.37534
4,45833,40.841409,-84.34178
...,...,...,...
43186,75034,33.143792,-96.83938
43187,41631,37.479754,-82.64922
43188,98362,48.097144,-123.40628
43189,83212,42.514471,-112.54626


In [13]:
#merge dataframes - now every org should have a corresponding latlong
new_org_col_df = pd.merge(new_zip3_df, org2_df,  how= "inner", on=["address_postcode", "address_postcode"])

In [14]:
new_org_col_df

,address_postcode,Latitude,Longitude,_id,index,Unnamed: 0,id,name,email,phone,...,hours_saturday,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,34611,28.564167,-82.416515,600c8d8c9129859f3e7d9a53,6495,6495,FL1360,"FurEver Friendz, Inc",debra.fureverfriendz@gmail.com,(727) 992-2372,...,NaN,NaN,If interested in adopting any of our FurEver F...,NaN,https://www.facebook.com/fureverfriendzinc/?re...,NaN,NaN,NaN,NaN,fl1360
1,36201,33.658803,-85.851780,600c8d8c9129859f3e7d8d11,3101,3101,AL75,The Animal Shelter - Calhoun County Humane Soc...,calhouncountyhumanesociety@gmail.com,(256) 405-6680,...,noon - 3,closed,Adopters are encouraged to visit the shelter t...,https://drive.google.com/file/d/1-Wp3VonqD_7-1...,https://www.facebook.com/Calhoun-County-Humane...,NaN,NaN,NaN,NaN,al75
2,94564,37.997509,-122.292080,600c8d8c9129859f3e7d9698,5540,5540,CA2819,It's A New Day Animal Rescue,itsanewdayanimalrescue@gmail.com,NaN,...,NaN,NaN,NaN,https://itsanewdayanimalrescue.org/adopt/adopt...,https://www.facebook.com/ItsANewDayAnimalRescue/,NaN,NaN,NaN,NaN,ca2819
3,94564,37.997509,-122.292080,600c8d8c9129859f3e7d9702,5646,5646,CA2819,It's A New Day Animal Rescue,itsanewdayanimalrescue@gmail.com,NaN,...,NaN,NaN,NaN,https://itsanewdayanimalrescue.org/adopt/adopt...,https://www.facebook.com/ItsANewDayAnimalRescue/,NaN,NaN,NaN,NaN,ca2819
4,11757,40.690049,-73.374420,600c8d8c9129859f3e7d8690,1436,1436,NY1499,"Pawsitive Possibilities Rescue, Inc.",info@pawsitiverescue.org,(631) 553-9774,...,NaN,NaN,Prior to adopting an animal these are some of ...,https://pawsitiverescue.org/dog-adoption-appli...,https://www.facebook.com/pawsitivepossibilitie...,NaN,NaN,https://www.instagram.com/pawsitivepossibiliti...,NaN,ny1499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8701,19380,39.983108,-75.593320,600c8d8c9129859f3e7da1b4,8384,8384,PA174,Brandywine Valley SPCA - Pennsylvania,info@bvspca.org,(610) 692-6113,...,11 am - 5 pm,11 am - 5 pm,NaN,NaN,http://www.facebook.com/BrandywineSPCA/,https://twitter.com/BrandywineSPCA,http://www.youtube.com/adoptccspca/,http://www.instagram.com/brandywinespca/,NaN,pa174
8702,19567,40.373285,-76.200540,600c8d8c9129859f3e7d8e4f,3419,3419,PA748,Karni Mata Rat Rescue,karnimataratrescue@hotmail.com,(484) 273-1179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pa748
8703,10303,40.629448,-74.162390,600c8d8c9129859f3e7d8d6f,3195,3195,NY1488,"For Blake's Sake Rescue, Inc.",forblakessakerescue@gmail.com,(732) 413-5006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ny1488
8704,44626,40.695377,-81.381020,600c8d8c9129859f3e7d9614,5408,5408,OH632,SAVE A MOM PREGNANT DOG RESCUE,saveamomrescue@aol.com,(330) 866-5405,...,NaN,NaN,Visit our website at http://www.saveamomrescue...,NaN,NaN,NaN,NaN,NaN,NaN,oh632


In [15]:
# check for NaNs - don't need this now
# new_org_col_df[new_org_col_df.isna().any(axis=1)]

In [16]:
#dropna for fields where there is no lat or long because zip code not in the list: none found 
new_org_col_df['Latitude'].isnull().values.any()

False

In [17]:
# borrowed code from petpy_kl.ipynb to address redundant organizations if present 
# *** NOTE *** org_df, and therefore subsequently created org_df based dataframes, was created from the 
# mongo db collection that was originally created and cleaned up through running petpy_kl.ipynb. 
# However, after merging with zip codes for some reason, new organization "name" duplicates seem to have
# emerged.   

unique_ids = new_org_col_df["id"].unique()
print (unique_ids)

['FL1360' 'AL75' 'CA2819' ... 'NY1488' 'OH632' 'TX306']


In [18]:
# checking list length
len_org = len(new_org_col_df)
len_org

8706

In [19]:
# checking number of unique names
len_name_unique = len(new_org_col_df["name"].unique())
len_name_unique

7626

In [20]:
# checking number of unique ids (column 1)
len_id_unique = len(new_org_col_df["id"].unique())
len_id_unique

7745

In [21]:
#review column headers: 
column_list = list(new_org_col_df.columns) 
column_list

['address_postcode',
 'Latitude',
 'Longitude',
 '_id',
 'index',
 'Unnamed: 0',
 'id',
 'name',
 'email',
 'phone',
 'url',
 'website',
 'mission_statement',
 'distance',
 'address_address1',
 'address_address2',
 'address_city',
 'address_state',
 'address_country',
 'hours_monday',
 'hours_tuesday',
 'hours_wednesday',
 'hours_thursday',
 'hours_friday',
 'hours_saturday',
 'hours_sunday',
 'adoption_policy',
 'adoption_url',
 'socialmedia_facebook',
 'socialmedia_twitter',
 'socialmedia_youtube',
 'socialmedia_instagram',
 'socialmedia_pinterest',
 'organization_id']

In [22]:
# send df to csv - commented out since csv is already created
# new_org_col_df.to_csv("organizations2_mongo.csv", index=False)

In [23]:
org_mongo_dict2 = new_org_col_df.to_dict("records")

In [25]:
db.org_new.insert_many(org_mongo_dict2)